In [29]:
%cd /code

/code


This notebook is ran in a docker container where the project directory (i.e. same directory as README.md) is located in `/code`, which is set above. If you run locally you'll need to set the path of your project directory accordingly.

The `load_dotenv` function below loads all the variables found in the `.env` file as environment variables. You must have a `.env` file located in the project directory containing your OpenAI API key, in the following format.

```
OPENAI_API_KEY=sk-...
```

In [30]:
from dotenv import load_dotenv
load_dotenv()

True

---

# Big Picture Examples

In [51]:
import chromadb
from chromadb.api.models.Collection import Collection
from llm_chain.base import Document

client = chromadb.Client()
collection = client.create_collection("test")


def add_embeddings(docs: list[Document]) -> None:
    embeddings = [x.embedding for x in docs]
    metadatas = [x.metadata for x in docs]
    documents = [x.content for x in docs]
    collection.add(
        embeddings=embeddings,
        metadatas=metadatas,
        documents=documents,
        ids=[str(x) for x in range(len(docs))],
    )


def search_embeddings(doc: Document, n_results: int = 3) -> list[Document]:
    query_result = collection.query(
        query_embeddings=[doc.embedding],
        n_results=n_results,
    )
    return query_result

In [53]:
from typing import get_type_hints

def add(a: int, b: int) -> int:
    return a + b

hints = get_type_hints(add)
hints

{'a': int, 'b': int, 'return': int}

In [50]:
[str(x) for x in range(len([1, 1]))]

['0', '1']

---

# OpenAI Chat

## Simple example showing history and usages/costs

In [31]:
from llm_chain.models import OpenAIChat

chat = OpenAIChat(model_name='gpt-3.5-turbo', temperature=0)
response = chat("Hi, my name is Shane.")
response

'Hello Shane! How can I assist you today?'

In [32]:
# the `history` property contains a list of `MessageMetaData` objects for each message (i.e.
# prompt & response) which contains usage/cost data for that message.
chat.history

[MessageMetaData(prompt='Hi, my name is Shane.', response='Hello Shane! How can I assist you today?', metadata={'model_name': 'gpt-3.5-turbo'}, prompt_tokens=26, response_tokens=10, total_tokens=36, cost=7.2e-05)]

In [33]:
# You can get the last MessageMetaData via: 
print(f"MessageMetaData: {chat.previous_message}")
# Or you can get the last prompt/response
print(f"previous prompt: {chat.previous_prompt}")
print(f"previous response: {chat.previous_response}")

MessageMetaData: prompt='Hi, my name is Shane.' response='Hello Shane! How can I assist you today?' metadata={'model_name': 'gpt-3.5-turbo'} prompt_tokens=26 response_tokens=10 total_tokens=36 cost=7.2e-05
previous prompt: Hi, my name is Shane.
previous response: Hello Shane! How can I assist you today?


In [34]:
# the model object tracks usage/cost data across all messages  
def print_usage(model: OpenAIChat):
    usage = f"""
    Total Cost: ${model.total_cost:.6f}
    Total Tokens: {model.total_tokens:,}
    Total Prompt Tokens: {model.total_prompt_tokens:,}
    Total Response Tokens: {model.total_response_tokens:,}
    """
    print(usage)

In [35]:
print_usage(model=chat)


    Total Cost: $0.000072
    Total Tokens: 36
    Total Prompt Tokens: 26
    Total Response Tokens: 10
    


In [36]:
# you can also see the exact messages sent to ChatGPT
chat._previous_memory

[{'role': 'system', 'content': 'You are a helpful assistant.'},
 {'role': 'user', 'content': 'Hi, my name is Shane.'}]

In [37]:
response = chat("Do you remember my name?")
response

'Yes, your name is Shane.'

In [38]:
chat.history

[MessageMetaData(prompt='Hi, my name is Shane.', response='Hello Shane! How can I assist you today?', metadata={'model_name': 'gpt-3.5-turbo'}, prompt_tokens=26, response_tokens=10, total_tokens=36, cost=7.2e-05),
 MessageMetaData(prompt='Do you remember my name?', response='Yes, your name is Shane.', metadata={'model_name': 'gpt-3.5-turbo'}, prompt_tokens=52, response_tokens=7, total_tokens=59, cost=0.000118)]

In [39]:
# you can also see the exact messages sent to ChatGPT
chat._previous_memory

[{'role': 'system', 'content': 'You are a helpful assistant.'},
 {'role': 'user', 'content': 'Hi, my name is Shane.'},
 {'role': 'assistant', 'content': 'Hello Shane! How can I assist you today?'},
 {'role': 'user', 'content': 'Do you remember my name?'}]

In [40]:
# You can get the last MessageMetaData via: 
print(f"MessageMetaData: {chat.previous_message}")
# Or you can get the last prompt/response
print(f"previous prompt: {chat.previous_prompt}")
print(f"previous response: {chat.previous_response}")

MessageMetaData: prompt='Do you remember my name?' response='Yes, your name is Shane.' metadata={'model_name': 'gpt-3.5-turbo'} prompt_tokens=52 response_tokens=7 total_tokens=59 cost=0.000118
previous prompt: Do you remember my name?
previous response: Yes, your name is Shane.


In [41]:
print_usage(model=chat)


    Total Cost: $0.000190
    Total Tokens: 95
    Total Prompt Tokens: 78
    Total Response Tokens: 17
    


---

## Memory

The `OpenAIChat` model has a `memory_strategy` parameter and takes a `MemoryBuffer` class. A `MemoryBuffer` class is a callable that takes a `list[MessageMetaData]` (i.e. from the `model.history` property) and also returns a `list[MessageMetaData]` serving as the model's memory (i.e. a list containing the messages that will be sent to the model along with the new prompt). This allows the end user to easily define a memory strategy of their own (e.g. keep the first message and the last `n` messages).

One Example of a `MemoryBuffer` is a `MemoryBufferMessageWindow` class where you can specify the last `n` messages that you want to keep.

In [42]:
from llm_chain.models import OpenAIChat
from llm_chain.memory import MemoryBufferMessageWindow

chat = OpenAIChat(
    model_name='gpt-3.5-turbo',
    temperature=0,
    memory_strategy=MemoryBufferMessageWindow(last_n_messages=0),  # no memory
)
response = chat("Hi, my name is Shane.")
response

'Hello Shane! How can I assist you today?'

In [43]:
# NOTE: since we created a new OpenAIChat object, the costs/usage are reset
print_usage(model=chat)


    Total Cost: $0.000072
    Total Tokens: 36
    Total Prompt Tokens: 26
    Total Response Tokens: 10
    


In [44]:
# you can also see the exact messages sent to ChatGPT
chat._previous_memory

[{'role': 'system', 'content': 'You are a helpful assistant.'},
 {'role': 'user', 'content': 'Hi, my name is Shane.'}]

In [45]:
response = chat("Do you remember my name?")
response

"I'm sorry, but as an AI language model, I don't have the ability to remember specific information about individual users. However, I'm always here to assist you with any questions or tasks you may have."

In [46]:
# we still have access to the full history, but the ChatGPT didn't use any of it.
chat.history

[MessageMetaData(prompt='Hi, my name is Shane.', response='Hello Shane! How can I assist you today?', metadata={'model_name': 'gpt-3.5-turbo'}, prompt_tokens=26, response_tokens=10, total_tokens=36, cost=7.2e-05),
 MessageMetaData(prompt='Do you remember my name?', response="I'm sorry, but as an AI language model, I don't have the ability to remember specific information about individual users. However, I'm always here to assist you with any questions or tasks you may have.", metadata={'model_name': 'gpt-3.5-turbo'}, prompt_tokens=25, response_tokens=43, total_tokens=68, cost=0.000136)]

In [47]:
# you can also see the exact messages sent to ChatGPT
chat._previous_memory

[{'role': 'system', 'content': 'You are a helpful assistant.'},
 {'role': 'user', 'content': 'Do you remember my name?'}]

In [48]:
# NOTE: since we created a new OpenAIChat object, the costs/usage are reset
print_usage(model=chat)


    Total Cost: $0.000208
    Total Tokens: 104
    Total Prompt Tokens: 51
    Total Response Tokens: 53
    


---